In [3]:
import pandas as pd
import numpy as np
import string
import json

In [55]:
# static config
primary_key = "Code"
locations = [
    {
        "column": "Woreda",
        "name": "level1"
    },
    {
        "column": "Kebele",
        "name": "level2"
    }
]
latlong = ["Latitude", "Longitude"]
charts = []
'''
charts = [
    {
        "name": "Type of Water Points",
        "column": "Is the water point currently functional?",
        "action": "count",
        "type": "stack",
        "value": [
            'Functional in use',
            'Non-functional'
        ],
        "and": None,
        "and_value": None,
    }
]
'''

## Table config
table = []
'''
table = [
    {
        "name": "Summary Data",
        "type": "summary",
        "indicators": [
            {
                "name": "Sub-County",
                "column": "Kebele",
                "action": "distinct",
                "type": "string",
                "value": None,
                "and": None,
                "and_value": None,
            },
            {
                "name": "Total Villages",
                "column": "Village",
                "action": "group-count",
                "type": "number",
                "value": None,
                "and": None,
                "and_value": None
            },
            {
                "name": "Number of Functional Waterpoints",
                "column": "Is the water point currently functional?",
                "action": "group-count",
                "type": "number",
                "value": None,
                "and": None,
                "and_value": None,
            }
        ],
    }
]
'''
marker = {
    "name" : "Water Supply System (WSS) Name",
    "color": [],
    "detail": [
        {
            "name": "Year Commisioned",
            "column": "Year Commisioned",
            "action": "select",
            "type": "number",
            "value": None,
            "and": None,
            "and_value": None,
        },
        {
            "name": "Source Type",
            "column": "Water Source Type",
            "action": "select",
            "type": "string",
            "value": None,
            "and": None,
            "and_value": None,
        }
    ]
}
'''
main = {
    'name':'Non-Functional Facility',
    'column': 'Water point Name',
    'select': {
        'column': 'Is the water point currently functional?',
        'value': 'Non-functional'
    },
    'indicators': [{
        'name': 'Number of HHs',
        'column': 'Estimate the number of households who use this water point as their source of water'
    },{
        'name': 'Type of Use',
        'column': 'What is the main purpose for which people use water from this water point?'
    },{
        'name': 'Nature of Technical Breakdown',
        'column': 'Please explain any other observable technical problems that require repairs'
    },{
        'name': 'WUC/WSC Exists',
        'column': 'Is there a WUC/ WSC for this water point?'
    },{
        'name': '% of Community Paying Tariffs',
        'column': '% of Community Paying Tariffs'
    }]
}
'''

"\nmain = {\n    'name':'Non-Functional Facility',\n    'column': 'Water point Name',\n    'select': {\n        'column': 'Is the water point currently functional?',\n        'value': 'Non-functional'\n    },\n    'indicators': [{\n        'name': 'Number of HHs',\n        'column': 'Estimate the number of households who use this water point as their source of water'\n    },{\n        'name': 'Type of Use',\n        'column': 'What is the main purpose for which people use water from this water point?'\n    },{\n        'name': 'Nature of Technical Breakdown',\n        'column': 'Please explain any other observable technical problems that require repairs'\n    },{\n        'name': 'WUC/WSC Exists',\n        'column': 'Is there a WUC/ WSC for this water point?'\n    },{\n        'name': '% of Community Paying Tariffs',\n        'column': '% of Community Paying Tariffs'\n    }]\n}\n"

In [63]:
df = pd.read_csv('eth_wp_data.csv')

In [64]:
df["Code"] = df.index + 1
df["Code"] = df.Code.astype(str).apply(lambda x: x.zfill(3)) +"-"+ df["Woreda"].str.lower() +"-"+ df["Kebele"].str.lower()

In [65]:
# Move second last column to be first
cols = list(df.columns)
cols = [cols[-1]] + cols[:-2]
df = df[cols]

In [66]:
# Fill empty Integer values
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df_num = df.select_dtypes(include=numerics).drop(columns=['Latitude', 'Longitude'])
df_num = df_num.fillna(0.0).astype(np.int32)
df[list(df_num)] = df_num
df['Yield'] = 0

In [67]:
# Replace Empty string values
df_str = df.select_dtypes(include=['object']).fillna("")
df[list(df_str)] = df_str

In [68]:
# Generate Settings
chars = list(string.ascii_uppercase)
chars_col = chars + [x+y for x in chars for y in chars]
keyname = lambda x,y: {a:y[b] if (a != 'data_point_id') else 'data_point_id' for b, a in enumerate(x)}

columns_length = len(list(df))
column_names = list(df)

In [69]:
### Generate configs
index = chars_col[:columns_length]
configs = keyname(index, column_names)

primary_key_config = "";
marker_config = {}
latlong_config = {}
name_config = {}
for key in configs:
    if(configs[key].lower() == primary_key.lower()):
        primary_key_config = key
    if(configs[key].lower() in latlong):
        latlong_config.update({configs[key].lower(): key})
    if(configs[key].lower() == marker['name'].lower()):
        marker_config = { "name": key }

### Locations
loc_config = {}
for loc in locations:
    column_index = ""
    for key in configs:
        if loc['column'].lower() == configs[key].lower():
            column_index = key
    loc_config.update({loc['name'].lower(): column_index})

### Marker Color
marker_color = [];
for mk in marker['color']:
    column = None
    andColumn = None
    for key in configs:
        if configs[key].lower() == mk['column'].lower():
            column = key
        if mk['and'] != None and configs[key].lower() == mk['and'].lower():
            andColumn = key
    marker_color.append({
        "name": mk['name'],
        "column": column,
        "action": mk['action'],
        "type": mk['type'],
        "value": mk['value'],
        "and": andColumn,
        "and_value": mk['and_value'],
        "color": mk['color']
    })
marker_config['color'] = marker_color

#### Marker Detail
marker_detail = []
for mk in marker['detail']:
    column = None
    andColumn = None
    for key in configs:
        if configs[key].lower() == mk['column'].lower():
            column = key
        if mk['and'] != None and configs[key].lower() == mk['and'].lower():
            andColumn = key
    marker_detail.append({
        "name": mk['name'],
        "column": column,
        "action": mk['action'],
        "type": mk['type'],
        "value": mk['value'],
        "and": andColumn,
        "and_value": mk['and_value']
    })
marker_config['detail'] = marker_detail

### Chart
chart_config = []
for ch in charts:
    column = None
    andColumn = None
    for key in configs:
        if configs[key].lower() == ch['column'].lower():
            column = key
        if ch['and'] != None and configs[key].lower() == ch['and'].lower():
            andColumn = key
    chart_config.append({
        "name": ch['name'],
        "column": column,
        "action": ch['action'],
        "type": ch['type'],
        "value": ch['value'],
        "and": andColumn,
        "and_value": ch['and_value'],
    })

#### Table
table_config = []
for tb in table:
    tb_indicators = []
    for ind in tb['indicators']:
        column = None
        andColumn = None
        for key in configs:
            if configs[key].lower() == ind["column"].lower():
                column = key
            if ind["and"] != None and configs[key].lower() == ind['and'].lower():
                andColumn = key
        tb_indicators.append({
            "column": column,
            "name": ind['name'],
            "action": ind['action'],
            "type": ind['type'],
            "value": ind['value'],
            "and": andColumn,
            "and_value": ind['and_value'],
        })
    table_config.append({
        "name": tb['name'],
        "type": tb['type'],
        "indicators": tb_indicators
    })

## Main Table
'''
main_config = {'name':main['name']}
main_selector = {}
indicators = []
for key in configs:
    if configs[key] == main['column']:
        main_config.update({'key':key})
    if configs[key] == main['select']['column']:
        main_selector.update({'key':key,'value':main['select']['value']})
    for i in main['indicators']:
        if configs[key] == i['column']:
            indicators.append({'name':i['name'],'key':key})
main_config.update({'indicators':indicators,'select':main_selector})
'''

configs['primary_key'] = primary_key_config
configs['marker'] = marker_config
configs['locations'] = loc_config
configs['latlong'] = latlong_config
configs['table'] = table_config

In [70]:
df.columns = index

In [71]:
configs

{'A': 'Code',
 'B': 'Woreda',
 'C': 'Kebele',
 'D': 'Village',
 'E': 'Site Name',
 'F': 'Latitude',
 'G': 'Longitude',
 'H': 'Water Source Type_1',
 'I': 'Water Source Type',
 'J': 'Functionality Status',
 'K': 'Souce of Energy',
 'L': 'Estimated Number of Users',
 'M': 'Hand Pump Type',
 'N': 'Organisation that installed the Water Pump',
 'O': 'Year Commisioned',
 'P': 'Reason for Inventory',
 'Q': 'Functional Taps In The Scheme',
 'R': 'Number of Non FunctionalTaps In The Scheme',
 'S': 'Depth',
 'T': 'Yield',
 'primary_key': 'A',
 'marker': {'color': [],
  'detail': [{'name': 'Year Commisioned',
    'column': 'O',
    'action': 'select',
    'type': 'number',
    'value': None,
    'and': None,
    'and_value': None},
   {'name': 'Source Type',
    'column': 'I',
    'action': 'select',
    'type': 'string',
    'value': None,
    'and': None,
    'and_value': None}]},
 'locations': {'level1': 'B', 'level2': 'C'},
 'latlong': {},
 'table': []}

In [72]:
data = list(df.T.to_dict().values())
data[:2]

[{'A': '001-shashemene-hagugeta keni',
  'B': 'Shashemene',
  'C': 'Hagugeta Keni',
  'D': 'Qali',
  'E': 'Qali',
  'F': 7.250937,
  'G': 38.62908,
  'H': 'Hand dug Well fitted with pump or windlass',
  'I': 'Hand dug Well',
  'J': 'Functional',
  'K': 'Manual operation',
  'L': '312',
  'M': 'Afridev',
  'N': 'CDA',
  'O': 2009,
  'P': '',
  'Q': 0,
  'R': 0,
  'S': 0,
  'T': 0},
 {'A': '002-shashemene-wetera shegule',
  'B': 'Shashemene',
  'C': 'Wetera Shegule',
  'D': 'Ardano',
  'E': 'Ardano',
  'F': 7.212806,
  'G': 38.7258,
  'H': 'Hand dug Well fitted with pump or windlass',
  'I': 'Hand dug Well',
  'J': 'Functional',
  'K': 'Manual operation',
  'L': '180',
  'M': 'Afridev',
  'N': '',
  'O': 2000,
  'P': '',
  'Q': 0,
  'R': 0,
  'S': 0,
  'T': 0}]

In [74]:
df.to_json('../resources/js/data/eth_wp_data.json', orient="records", indent=2)
with open('../resources/js/data/eth_wp_config.json', 'w') as outfile:
    json.dump(configs, outfile)